# Issue Trend Analysis and Issue Tracking
##### Copyright (C) CS474 Team #4 

----------------------------------------------------

### Imports & Statics

In [1]:
import csv, pickle
import utils
from utils import Corpus, Issue, Extractor, IssueModel, EventModel
import os, sys
import warnings

warnings.filterwarnings('ignore')

years = [2015, 2016, 2017]
num_issues = 50
num_events = 50
num_keywords = 10

----------------------------------------------------

## Part 0: Preprocessing

### Clean articles: lemmatize, remove stopwords (Already Done)
**_Caution!_ This overwrites dumps/cleaned.bin file. ** (Involves multiprocessing)

In [2]:
# utils.clean_articles()

### Detect Entities: IBM Watson NLU (Already Done)
**_Caution!_ This overwrites dumps/cleaned_watson.bin file. ** (Involves multiprocessing and Watson API calls)

In [3]:
# utils.build_watson()

## Part 1: Issue Trend Analysis

#### Initialize Corpuses

In [4]:
corpus = {}
for year in years:
    corpus[year] = Corpus(year=year-2015)

#### Build Corpuses: Load cleaned articles, build phrasers, dictionary, and BOWs

In [5]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_corpus()
    print("Corpus "+str(year)+" Done\n")


Corpus 2015:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 7.40 seconds
trigram train finished! 12.16 seconds
building dictionary...
building bag of words...


100%|██████████| 7155/7155 [00:03<00:00, 2038.94it/s]


Corpus 2015 Done

Corpus 2016:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 7.98 seconds
trigram train finished! 12.99 seconds
building dictionary...
building bag of words...


100%|██████████| 7480/7480 [00:03<00:00, 1881.52it/s]


Corpus 2016 Done

Corpus 2017:
building corpus...
collecting articles...
building phrasers...
bigram train finished! 8.76 seconds
trigram train finished! 14.80 seconds
building dictionary...
building bag of words...


100%|██████████| 9117/9117 [00:04<00:00, 2159.72it/s]


Corpus 2017 Done



#### Extract Keywords from each Article using tf-ifd

In [6]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_tfidf()
    corpus[year].extractor = Extractor(corpus[year])
    corpus[year].extractor.extract(k=num_keywords)
    print("Corpus "+str(year)+" Done\n")

Corpus 2015:
building tf-idf model...
tfidf finished! 0.07 seconds
extracting keywords...


100%|██████████| 7155/7155 [00:04<00:00, 1682.06it/s]


Corpus 2015 Done

Corpus 2016:
building tf-idf model...
tfidf finished! 0.07 seconds
extracting keywords...


100%|██████████| 7480/7480 [00:04<00:00, 1680.44it/s]


Corpus 2016 Done

Corpus 2017:
building tf-idf model...
tfidf finished! 0.08 seconds
extracting keywords...


100%|██████████| 9117/9117 [00:05<00:00, 1712.40it/s]

Corpus 2017 Done



#### Save Corpuses

In [7]:
utils.save(corpus, filename='corpus_ready.bin')

saving corpus_ready.bin...
corpus_ready.bin saved


#### Load corpus

In [2]:
corpus = utils.load(filename='corpus_ready.bin')

loading corpus_ready.bin...
corpus_ready.bin loaded


#### Build LDA model, cluster articles into issues

In [4]:
for year in years:
    print("Corpus "+str(year)+":")
    corpus[year].build_lda(num_topics=num_issues)
    corpus[year].issue_model = IssueModel(corpus=corpus[year], model=corpus[year].lda)
    corpus[year].issue_model.build_issues()
    print("Corpus "+str(year)+" Done\n")

Corpus 2015:
building LDA model...
LDA finished! 8.86 seconds
building issues...


100%|██████████| 7155/7155 [00:06<00:00, 1190.21it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 3302.34it/s]


Corpus 2015 Done

Corpus 2016:
building LDA model...
LDA finished! 9.58 seconds
building issues...


...

Corpus 2016 Done

Corpus 2017:
building LDA model...
LDA finished! 9.99 seconds
building issues...


100%|██████████| 9117/9117 [00:06<00:00, 1316.36it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 2003.18it/s]

Corpus 2017 Done



#### Save Corpuses

In [4]:
utils.save(corpus, filename='corpus_done.bin')

saving corpus_done.bin...
corpus_done.bin saved


#### Init Issues (for Part 2)

In [5]:
issues = []
for year in years:
    issue_model = corpus[year].issue_model
    top_issue_id = issue_model.sorted_issues[0][0]
    issues.append(Issue(articles=issue_model.issues[top_issue_id], keywords=issue_model.keywords[top_issue_id]))

#### Save Issues (for Part 2)

In [6]:
utils.save(issues, filename='issues_init.bin')

saving issues_init.bin...
issues_init.bin saved


----------------------------------------------------

### Show Results

#### Load corpus

In [17]:
corpus = utils.load(filename='corpus_done.bin')

loading corpus_done.bin...
corpus_done.bin loaded


#### Select year to show

In [19]:
show_year = 2017

#### Show top trending issues  

In [20]:
corpus[show_year].issue_model.show_top_issues()

ID:  24 Score: 1907.37 N:  744 Keywords:  Trump, N._Korea, S._Korea, Tillerson, NK, Mattis, THAAD, White_House, dialogue, envoy
ID:  28 Score: 1502.28 N:  560 Keywords:  N._Korea, NK, S._Korea, missile_launch, ICBM, DPRK, sanction, missile_test, Hwasong-, missile
ID:  22 Score: 948.06 N:  295 Keywords:  Ahn, Hong, People_Party, Yoo, candidate, Bareun_Party, primary, Election, Hwang, presidential_race
ID:   9 Score: 880.82 N:  385 Keywords:  Choi, Park_lawyer, trial, prosecution, Court, Woo, Park_aide, Park, arrest_warrant, arrest
ID:  19 Score: 789.48 N:  126 Keywords:  S._Korea, NK, aid, inter-Korean_exchange, Malaysia, N._Korean, assistance, humanitarian_assistance, complex, UN
ID:  38 Score: 646.02 N:  115 Keywords:  comfort_woman, statue, Dokdo, girl_statue, woman_statue, claim_Dokdo, sex_slave, sex_slavery, islet, sexual_slavery
ID:  23 Score: 615.94 N:   89 Keywords:  Incheon, victim, accomplice, murder, suspect, man, Police, jail_term, get_jail, bomb
ID:  21 Score: 612.78 N:  11

#### Show Articles from Top Issues 

In [5]:
corpus[show_year].issue_model.show_issues(k=5)

ID:   5 Score: 1109.61 N:  539 Keywords:  Saenuri, bill, impeachment, Moon, Ahn, Chung, Minjoo, Minjoo_Party, constituency, Opposition_party
	 0 	 Former leader quits opposition party
	 1 	 [Newsmaker] Park’s lame duck deadline looms
	 2 	 Constitutional reform debate resurfaces
	 3 	 General elections mired in uncertainty without constituencies
	 4 	 Park asks political parties to embrace reform
ID:  14 Score: 946.36 N:  275 Keywords:  N._Korea, sanction, N_K, S._Korea, human_right, Obama, US, NK, resolution, White_House
	 0 	 Quake in North Korea suspected to be 'explosion': report
	 1 	 North Korea announces successful test of hydrogen bomb
	 2 	 China party paper urges North Korea to change 'nuclear path'
	 3 	 Park, Obama talk over N.K. nuke test
	 4 	 Park, Obama agree to closely work together to adopt strong U.N. sanctions against North
ID:   1 Score: 767.23 N:  187 Keywords:  Trump, comfort_woman, Japan, foundation, S._Korea, sex_slavery, Iran, victim, sexual_slavery, sex_slave

#### View LDA Model

In [26]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [24]:
pyLDAvis.gensim.prepare(corpus[show_year].lda, corpus[show_year].get_bows(), corpus[show_year].dict)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24     0.185430  0.060264       1        1  13.697927
28     0.189284  0.028263       2        1   9.839084
22    -0.061727  0.161248       3        1   6.148484
9     -0.155755  0.148024       4        1   5.585011
19     0.148338 -0.000279       5        1   3.783947
38     0.019873 -0.079133       6        1   3.333905
14     0.088138 -0.121769       7        1   3.267164
21    -0.078592 -0.125563       8        1   2.522198
2     -0.014070 -0.086329       9        1   2.418282
4     -0.120544  0.194655      10        1   2.406323
23    -0.103869 -0.062514      11        1   2.377939
8     -0.074249  0.020735      12        1   2.372035
17     0.174111  0.090937      13        1   2.131275
48    -0.035772  0.121611      14        1   2.080758
10     0.097499 -0.000876      15        1   2.021873
49    -0.034328 -0.129521      16        1   1.939103
37     0.006956 -0.028459      17        1   1.799561
35    -0.004489 -0.114161      18        1   1.755271
29     0.119332  0.011937      19        1   1.702442
20    -0.071520 -0.065654      20        1   1.552740
18    -0.132074  0.006808      21        1   1.533495
6      0.020215 -0.120472      22        1   1.490291
16     0.117961 -0.002233      23        1   1.454999
34    -0.026130  0.027025      24        1   1.446244
7      0.142360  0.081728      25        1   1.378315
46    -0.077440 -0.101138      26        1   1.317735
13     0.110915 -0.033242      27        1   1.243706
25     0.005718 -0.135544      28        1   1.196827
0      0.000595  0.081261      29        1   1.174513
41     0.161652  0.003379      30        1   1.067852
45    -0.061534  0.070365      31        1   0.982310
26     0.012873  0.069615      32        1   0.966353
3      0.044952  0.060182      33        1   0.936444
5      0.032559  0.034901      34        1   0.871822
40     0.009135 -0.086446      35        1   0.830469
42    -0.082204 -0.067468      36        1   0.813934
12    -0.048085 -0.046666      37        1   0.795813
32    -0.085620 -0.079565      38        1   0.794625
43    -0.075215  0.028220      39        1   0.779339
30    -0.065100  0.043222      40        1   0.740631
27    -0.122672  0.083926      41        1   0.737891
39     0.025231  0.008719      42        1   0.688407
47    -0.066367 -0.044901      43        1   0.599559
44    -0.036546 -0.008370      44        1   0.573259
36    -0.020502 -0.036244      45        1   0.561586
33     0.003310  0.095417      46        1   0.554417
11    -0.005290  0.033350      47        1   0.498859
31    -0.026103 -0.025739      48        1   0.478052
15     0.020761  0.007094      49        1   0.400626
1     -0.051400  0.029399      50        1   0.356300, topic_info=      Category          Freq                  Term         Total  loglift  \
term                                                                        
7      Default  18353.000000           North_Korea  18353.000000  30.0000   
760    Default   8206.000000                  Moon   8206.000000  29.0000   
2598   Default   5366.000000                 China   5366.000000  28.0000   
88     Default   5393.000000          North_Korean   5393.000000  27.0000   
365    Default  12641.000000                    US  12641.000000  26.0000   
1357   Default   3847.000000               percent   3847.000000  25.0000   
120    Default   3607.000000              military   3607.000000  24.0000   
237    Default   2391.000000              S._Korea   2391.000000  23.0000   
87     Default   7686.000000                 North   7686.000000  22.0000   
3023   Default   3577.000000                 Japan   3577.000000  21.0000   
716    Default   3817.000000                 party   3817.000000  20.0000   
1292   Default   6870.000000                 Korea   6870.000000  19.0000   
614    Default   3853.000000                 Trump   3853.000000  18.00

In [27]:
# issues_lda_data = pyLDAvis.gensim.prepare(corpus[show_year].lda, corpus[show_year].get_bows(), corpus[show_year].dict)
# utils.save(issues_lda_data, 'issues_lda_data.bin')

saving issues_lda_data.bin...
issues_lda_data.bin saved


----------------------------------------------------

## Part 2: Issue Tracking

### Imports & Statics

In [6]:
import csv, pickle
import utils
from utils import Corpus, Issue, Extractor, IssueModel, EventModel
import os, sys
import warnings

warnings.filterwarnings('ignore')

years = [2015, 2016, 2017]
num_issues = 50
num_events = 50
num_keywords = 10

#### Load Issues

In [2]:
issues = utils.load('issues_init.bin')

loading issues_init.bin...
issues_init.bin loaded


#### Build Issues

In [3]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_issue()
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building issue...
building phrasers...
bigram train finished! 0.35 seconds
trigram train finished! 0.49 seconds
building dictionary...
building bag of words...


100%|██████████| 351/351 [00:00<00:00, 2260.23it/s]


Issue 1 Done

Issue 2:
building issue...
building phrasers...
bigram train finished! 0.82 seconds
trigram train finished! 1.22 seconds
building dictionary...
building bag of words...


100%|██████████| 539/539 [00:00<00:00, 1619.94it/s]


Issue 2 Done

Issue 3:
building issue...
building phrasers...
bigram train finished! 0.96 seconds
trigram train finished! 1.47 seconds
building dictionary...
building bag of words...


100%|██████████| 744/744 [00:00<00:00, 1841.75it/s]

Issue 3 Done



#### Extract keywords from each Article using tf-idf

In [4]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_tfidf()
    issue.extractor = Extractor(issue)
    issue.extractor.extract(k=num_keywords)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 351/351 [00:00<00:00, 1679.65it/s]


Issue 1 Done

Issue 2:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 539/539 [00:00<00:00, 1355.12it/s]


Issue 2 Done

Issue 3:
building tf-idf model...
tfidf finished! 0.01 seconds
extracting keywords...


100%|██████████| 744/744 [00:00<00:00, 1474.12it/s]

Issue 3 Done



#### Save Issues

In [6]:
utils.save(issues, filename='issues_ready.bin')

saving issues_ready.bin...
issues_ready.bin saved


#### Load Issues

In [94]:
issues = utils.load(filename='issues_ready.bin')

loading issues_ready.bin...
issues_ready.bin loaded


#### Build LDA model, cluster articles into events

In [5]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.build_lda(num_topics=num_events)
    issue.event_model = EventModel(issue=issue, model=issue.lda)
    issue.event_model.build_events(threshold=0.5)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building LDA model...
LDA finished! 0.66 seconds
building events...


100%|██████████| 351/351 [00:00<00:00, 1262.13it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 12995.92it/s]


Issue 1 Done

Issue 2:
building LDA model...
LDA finished! 1.18 seconds
building events...


...

Issue 2 Done

Issue 3:
building LDA model...
LDA finished! 1.49 seconds
building events...


100%|██████████| 744/744 [00:00<00:00, 1027.08it/s]


extracting keywords...


100%|██████████| 50/50 [00:00<00:00, 2281.67it/s]

Issue 3 Done



#### Divide events into set of independent events

In [10]:
threshold=[0.15, 0.15, 0.15]
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    issue.event_model.build_independents(threshold=threshold[i])
    issue.event_model.filter_events(k=5)
    issue.event_model.build_event_times()
    issue.event_model.build_sorted_independents()
    issue.event_model.build_event_details()
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
building event times...


100%|██████████| 5/5 [00:00<00:00, 1834.62it/s]


building event details...


100%|██████████| 5/5 [00:00<00:00, 1398.29it/s]


Issue 1 Done

Issue 2:
building event times...


...

Issue 2 Done

Issue 3:
building event times...


100%|██████████| 5/5 [00:00<00:00, 1185.77it/s]


building event details...


100%|██████████| 5/5 [00:00<00:00, 1232.46it/s]

Issue 3 Done



In [11]:
for i, issue in enumerate(issues):
    print("Issue "+str(i+1)+":")
    print(issue.event_model.sorted_independents)
    print("Issue "+str(i+1)+" Done\n")

Issue 1:
[[3, 11], [10, 6], [43]]
Issue 1 Done

Issue 2:
[[39], [15], [18], [25], [28]]
Issue 2 Done

Issue 3:
[[38], [29, 40, 7], [21]]
Issue 3 Done



#### Save Issues

In [21]:
utils.save(issues, 'issues_done.bin')

saving issues_done.bin...
issues_done.bin saved


----------------------------------------------------

### Show Result

#### Load Issues

In [2]:
issues = utils.load('issues_done.bin')

loading issues_done.bin...
issues_done.bin loaded


#### Select Issue

In [12]:
i = 2

#### Show Issue Keywords

In [13]:
print(', '.join([keyword[0] for keyword in issues[i].keywords]))

Trump, N._Korea, S._Korea, Tillerson, NK, Mattis, THAAD, White_House, dialogue, envoy


#### Show Top Events  

In [14]:
issues[i].event_model.show_top_events()

ID:  29 Score:  90.39 N:   49 Keywords:  Mattis, S._Korea, Pence, Vice_President, Guam, State_Department, Yun, security_chief, future_Seoul-Washington, cast_bright
ID:  21 Score:  86.39 N:   54 Keywords:  Yun, S._Korea, dialogue_table, NK_nuke, Dae_spokesman, bilateral_summit, UN_sanction, Chung, beneficial, mutually
ID:  40 Score:  73.43 N:   46 Keywords:  Mattis, foreign_minister, Kang, missile_launch, objective, S._Korea, pay_system, question, respond, Yun
ID:  38 Score:  70.61 N:   36 Keywords:  Mattis, Tillerson, military_option, NK, White_House, McMaster, diplomatic_effort, question_sincerity, hurt_Chinese, fruitless
ID:   7 Score:  70.19 N:   43 Keywords:  Tillerson, S._Korea, envoy, Kang, spokesperson, White_House, welcome, denuclearize, right_time, professor
ID:  12 Score:  55.72 N:   32 Keywords:  adviser, S._Korea, source, Korea-US_alliance, room, Moon, by-election, kilometer, Korea-US_FTA, initiative
ID:   1 Score:  51.28 N:   32 Keywords:  S._Korea, Republic_of_Korea, THAA

#### Show Articles from Top Events 

In [15]:
issues[i].event_model.show_events(k=5)

ID:  29 Score:  90.39 N:   49 Keywords:  Mattis, S._Korea, Pence, Vice_President, Guam, State_Department, Yun, security_chief, future_Seoul-Washington, cast_bright
	 0 	 S. Korea's top security official to visit US
	 1 	 Kerry warns of 'forceful ways' against N. Korea
	 2 	 Seoul welcomes US secretary nominee's suggestion of continued sanctions on N. Korea
	 3 	 Korea-Japan ties going beyond 'difficult history' also good for US: Victor Cha
	 4 	 Korea seeks high-level talks with US in Feb.: source
ID:  21 Score:  86.39 N:   54 Keywords:  Yun, S._Korea, dialogue_table, NK_nuke, Dae_spokesman, bilateral_summit, UN_sanction, Chung, beneficial, mutually
	 0 	 Controversy brews over lawmakers’ China visit amid THAAD row
	 1 	 New type of cyberattacks to rise in 2017: report
	 2 	 Resumption of Kaesong complex to be hotly debated amid sanctions regime
	 3 	 S. Korea, Japan, China to discuss N. Korea's cyber threats in Tokyo this week
	 4 	 Top diplomats of S. Korea, US to discuss NK nukes in

#### Show Issue Summary

In [16]:
issues[i].event_model.show_issue_summary(num_entities=10)

Issue Keywords: 
	 Trump, N._Korea, S._Korea, Tillerson, NK, Mattis, THAAD, White_House, dialogue, envoy

Events: 
	 38
	 29 -> 40 -> 7
	 21


Event ID:  38
Event Keywords: 
	 Mattis, Tillerson, military_option, NK, White_House, McMaster, diplomatic_effort, question_sincerity, hurt_Chinese, fruitless
Time: 2017-07-19
Entities: 
	North Korea, LOCATION
	US, LOCATION
	Pyongyang, LOCATION
	WASHINGTON, LOCATION
	China, LOCATION
	Donald Trump, PERSON
	President, JOBTITLE
	United States, LOCATION
	Rex Tillerson, PERSON
	South Korea, LOCATION

Event ID:  29
Event Keywords: 
	 Mattis, S._Korea, Pence, Vice_President, Guam, State_Department, Yun, security_chief, future_Seoul-Washington, cast_bright
Time: 2017-05-06
Entities: 
	North Korea, LOCATION
	South Korea, LOCATION
	US, LOCATION
	Seoul, LOCATION
	Donald Trump, PERSON
	Pyongyang, LOCATION
	United States, LOCATION
	China, LOCATION
	Rex Tillerson, PERSON
	Washington, LOCATION

Event ID:  40
Event Keywords: 
	 Mattis, foreign_minister, Kang, m

#### View LDA Model

In [22]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [23]:
pyLDAvis.gensim.prepare(issues[i].lda, issues[i].get_bows(), issues[i].dict)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
21     0.029373  0.015635       1        1  9.977802
29     0.033226  0.017498       2        1  8.687294
40     0.024608  0.016936       3        1  7.846583
7      0.032262  0.004904       4        1  6.476141
12     0.034301  0.004170       5        1  5.784591
38     0.047045 -0.024065       6        1  5.327897
41     0.037624 -0.022720       7        1  4.708272
1      0.030016  0.034834       8        1  4.682981
34     0.029228 -0.007501       9        1  4.069176
42     0.019840  0.013736      10        1  3.096823
15     0.020502  0.016794      11        1  2.977605
11     0.026284  0.004779      12        1  2.859726
4      0.033303 -0.038989      13        1  2.768466
6      0.033527 -0.018947      14        1  2.686346
23     0.021081  0.005867      15        1  2.491925
39     0.026987 -0.015562      16        1  2.417084
36     0.025282 -0.001379      17        1  2.112894
19     0.019018 -0.010221      18        1  1.868960
31     0.020535  0.018348      19        1  1.630069
44     0.013324  0.003832      20        1  1.601255
46     0.017118  0.015083      21        1  1.383837
16     0.011575  0.016341      22        1  1.329316
30     0.016118  0.001129      23        1  1.254459
20     0.009899  0.039933      24        1  1.207579
22     0.014919 -0.043737      25        1  1.058666
8      0.006768 -0.036846      26        1  0.956187
14     0.004414  0.021124      27        1  0.887661
33     0.010779  0.018704      28        1  0.758957
2      0.006178  0.020631      29        1  0.729291
28     0.022005 -0.022847      30        1  0.692435
0     -0.013792 -0.008498      31        1  0.598689
25    -0.017513 -0.010816      32        1  0.531059
26    -0.010277 -0.009410      33        1  0.467502
9     -0.013809  0.012076      34        1  0.466548
5     -0.003231  0.003575      35        1  0.460884
13     0.000184 -0.002721      36        1  0.424862
3     -0.003730 -0.025262      37        1  0.367400
18    -0.003953 -0.009807      38        1  0.340345
49    -0.027012 -0.005895      39        1  0.316787
32    -0.029818 -0.010251      40        1  0.282739
45    -0.031587  0.026324      41        1  0.250052
37    -0.016765  0.034354      42        1  0.235178
48    -0.071389  0.056272      43        1  0.154227
17    -0.058498 -0.019404      44        1  0.151141
10    -0.035474 -0.061034      45        1  0.148542
47    -0.061621 -0.007318      46        1  0.147732
24    -0.033255 -0.003627      47        1  0.128106
27    -0.051501 -0.001341      48        1  0.096790
35    -0.039406  0.006502      49        1  0.092622
43    -0.154695 -0.011184      50        1  0.008513, topic_info=     Category         Freq              Term        Total  loglift  logprob
term                                                                       
108   Default  4470.000000                US  4470.000000  30.0000  30.0000
60    Default  3610.000000       North_Korea  3610.000000  29.0000  29.0000
105   Default  2363.000000             Trump  2363.000000  28.0000  28.0000
89    Default  1616.000000       South_Korea  1616.000000  27.0000  27.0000
13    Default  1719.000000             China  1719.000000  26.0000  26.0000
59    Default  1476.000000             North  1476.000000  25.0000  25.0000
565   Default  1325.000000              talk  1325.000000  24.0000  24.0000
140   Default  1205.000000              also  1205.000000  23.0000  23.0000
1143  Default  1036.000000          N._Korea  1036.000000  22.0000  22.0000
77    Default  1014.000000             Seoul  1014.000000  21.0000  21.0000
4876  Default   962.000000              Moon   962.000000  20.0000  20.0000
778   Default   720.000000               say   720.000000  19.0000  19.0000
1839  Default   667.000000                NK   667.000000  18.0000  18.0000
424   Default   613.000000          official   613.000000  17

In [54]:
# events_lda_data = pyLDAvis.gensim.prepare(issues[i].lda, issues[i].get_bows(), issues[i].dict)
# utils.save(events_lda_data, 'events_lda_data.bin')

saving events_lda_data.bin...
events_lda_data.bin saved
